In [1]:
import math
import numpy as np
from numpy.linalg import norm
from math import sqrt
from matplotlib import pyplot as plt
from helper import rate_of_expansion, rate_of_bearing, bearing, simple_gaussian, broken_sigmoid, minimum_separation


In [14]:
# Generate trajectories for two agents
n = 1000 # number of frames
w = 4 # width of each agent
# Agent 0 passing in front of agent 1
x0 = np.zeros(n)
y0 = np.linspace(sqrt(w) - 10, sqrt(w), num=n)
# y0 = np.linspace(-10, 0, num=n)
p0_f = np.stack((x0, y0), axis=-1)
x1 = np.linspace(-sqrt(w) - 10, -sqrt(w), num=n)
# x1 = np.append(np.linspace(-9.9, -4.9, num=int(n/2)), np.linspace(-4.9, 0.1, num=int(n/2)))
y1 = np.zeros(n)
p1_f = np.stack((x1, y1), axis=-1)

# Agent 0 passing behind agent 1
p0_b = p0_f - [0, 2 * sqrt(w)]
p1_b = p1_f + [2 * sqrt(w), 0]



def collision_trajectory(bearing, side, spd1=1.0, w=0.8, r=10, Hz=100):
    '''
    This function produces near-collision trajectories of circular agents 0 and 1 with 
    a defined bearing angle. The speed of agent 1 is constant. The speed of agent 0 will 
    vary to guarantee near-collision. 
    
    Args:
        bearing (float, 0-90): The initial bearing angle between A and B in degree. Bearing 
            angle is defined as the angle between the line of sight and heading direction.        
        side (char): Agent 0 pass in front ('f'), or from behind ('b') agent 1. 
        spd1 (float): The speed of agent 1 in meter/second.
        w (float): The diameter of both agents in meters.
        r (float): The distance from agent 1 to origin of the coordinate system. 
        Hz (int): Number of Hz for the simulation.
        
    Return:
        traj0, traj1 (2-d np array): Near-collision trajectories of agent 1 and agent 0 in 
            meters.
        spd0 (float): The speed of agent 0 to achieve near-collision with agent 1.
    '''
    alpha = (180 - 2 * bearing) * 2 * math.pi / 360 # Central angle of line of sight
    p1 = (-r * np.sin(alpha), -r * np.cos(alpha))
    p0 = (0, -r)
    v1 = (spd1 * np.sin(alpha), spd1 * np.cos(alpha))
    dca, ttca = minimum_separation(p0, p1, v0, v1)
    
    return traj0, traj1, spd0







(0.8944271909999161, 1.5999999999999996)

In [3]:
v0 = np.tile([0, 1], (n, 1))
v1 = np.tile([1, 0], (n, 1))
    
    
d_beta_f = rate_of_bearing(p0_f, p1_f, v0) * 100
d_beta_b = rate_of_bearing(p0_b, p1_b, v0) * 100
beta_f = bearing(p0_f, p1_f, v0)
beta_b = bearing(p0_b, p1_b, v0)

d_beta_f_gaussian = simple_gaussian(d_beta_f, 0.1, 10)
d_beta_f_sigmoid = broken_sigmoid(d_beta_f, 0.001, 40)
d_beta_b_gaussian = simple_gaussian(d_beta_b, 0.1, 10);
d_beta_b_sigmoid = broken_sigmoid(d_beta_b, 0.001, 40)

d_theta = rate_of_expansion(p0_f, p1_f, v0, v1, 2, relative=False)
d_theta_relative = rate_of_expansion(p0_f, p1_f, np.tile([0, 1], (n, 1)), np.tile([1, 0], (n, 1)), 2, relative=True)




In [4]:
# Plot
%matplotlib qt
# plt.plot(d_beta_f, label='d_beta_f')
# plt.plot(d_theta_relative, label='d_theta_relative')

# Critical d bearing angle as a function of (relative) rate of expansion
plt.plot(d_theta_relative[0:860], d_beta_f[0:860])

# Gaussian function of d bearing angle
# plt.plot(d_beta_b_gaussian)
# plt.plot(d_theta * d_beta_b_gaussian * 100)
# plt.plot([(10 - np.sqrt(2))*100, (10 - np.sqrt(2))*100],[0, 0.3]) # the moment when bearing angle is 90
# plt.plot(x, simple_gaussian(x, 2, 0.1))
# plt.plot(x, broken_sigmoid(x, 1, 20))

plt.legend()

No handles with labels found to put in legend.
